# پروژه پایانی درس هوش مصنوعی و سیستم های خبره #
## محمدحسین ارسلان 98243005 ##
### پرژه شمارۀ یک ###

#### توضیحات: سعی شده است در طول این نوتبوک توضیحات کامل به منظور جایگزین برای
#### گزارش نوشته شود، همچنین در نقاطی سعی شده با چاپ کردن دیتافریم های حاصل و یا 
#### .لیست ها و متغیرها روند کار را نشان داده و ابهامی به جای نگذارم


# Import the importants

In [1]:
import pandas as pd
import numpy as np
import hazm
from __future__ import unicode_literals
from hazm import *

# Reading train data

### برای شروع نیاز به یک داده داریم که ما داده آموزشی قرار داده شده در کگل را دریافت میکنیم

In [2]:
df = pd.read_csv('train.csv')

# Thanks to hazm Library :)

### سپس یک  ابزار از کتابخانه هضم را به برنامه خود اضافه می کنیم و در طول پیش پردازش با آن سر و کار داریم
### این ابزار مسوول نرمالایز کردن جلات و حذف نیم فاصله ها و تشخیص یک سری حروف که برای جمع کردن کلمات 
### به کار می روند است.
### همچنین یک تابع از این کتابخانه برای توکن بندی کردن کلمات داریم که در تابع توکنایز خودمان از آن بهره برده ایم.

In [3]:
nrm = Normalizer() #some verbs in persian got two parts which are spaced, this module help us to find relative tokens which have space in middle

# Normalize, Tokenize and Cleaning data

### اصل کار اینجا و در سلول زیر است چرا که داده ها که گفته های ما هستند باید توکن بندی شوند و به
###  کمک این توکن ها باید پیشبینی لیبل صورت گیرد. از این رو ما با پیمایشی روی جملات یا کوئری ها
### داریم و هربار یک کوئری را به تابع توکنایز می دهیم و در ستونی جدید از دیتافریم آموزش و تست این
### خروجی تابع توکنایز که یک لیست از توکن های جمله نرمال شده پاس داده شده است قرار می دهیم و
### مجدد در تابع کا-فولد روی این ستون توکن ها که ستون جدید دیتافریممان است پیمایش کرده و روی هر
### لیست موجود در آن تابع تمیزکننده را اجرا می کنیم که هرسری بررسی می کند و فقط کلماتی را در لیست
### پاس داده شده نگه می دارد که در لیست اضافه (خودم تعریف کرده ام و یک حالت دیکشنری گونه دارد) 
### نباشد را در لیست توکن ها نگه می دارد و با این کار داده ها تمیز می شوند و توکن های هر جمله ارزش
### .معنایی بالایی پیدا می کنند

In [4]:
extra = ['؟','.','باشد','است','نیست','هست','،',',','؛',';','?','.','-','/','_'
         ,'با','از','یا','چه','چیه','کیه','چطوریه','چقدر','کی','این','ان','آن','اون'
         ,'برای','را','رو','تو','من','ما','توی','چرا','هر','و','چقدره','آیا','چند'
        ,'0','1','2','3','4','5','6','7','8','9','تا','به','!']
def tokenize(sentence):
    sentence = nrm.normalize(sentence)
    sentence = word_tokenize(sentence)
    return sentence
def clean(tokens):
    tokens = list(filter(lambda i : extra.count(i) == 0, tokens))
    return tokens


# Prediction function 
## (Based on naive bayes)

### در بخش زیر یک سری تابع که برای محاسبات الگوریتم مورد نظر نیاز است طراحی و پیاده سازی شده است
### تابع اصلی این بخش که در تابع کا-فولد هم صدا زده می شود، تابع پیشبینی است، این تابع به ازای هر لیستی
### از توکن های سطرهای داده تست صدا زده می شود. یکسری دیتافریم که مخصوص هر لیبل بود را نیز به آن
### پاس می دهیم. این تابع در درون خود احتمال اینکه لیست توکن های ما متعلق به هریک از کلاس ها باشند را
### به کمک 5 تابع محاسبه احتمال به دست می آورد و پس از مقایسه آن ها بایکدیگر، یک لیبل به ما می دهد که 
### .همان مقدار پیشبینی شده است
### در احتمال هایی که در 5 تابع یادشده باید محاسبه شوند مخرج کسر ها در هر لیبل مقدار ثابتی است و متشکل
### از مجموع تعداد توکن های متمایز کل داده آموزشی و مجموع کل توکن های داده های آن لیبل است که برای 
### .جلوگیری از تکرار آن ها را داخل تابع پردیکت محاسبه کرده و به توابع احتمال پاس می دهیم

In [5]:
def probability_label1(p,tokens,l,unique,label_words):
    probe = p
#     prob *= for each token => ((tekrar kalame dar jomlat in label + 1) / (unique words in train set + all words in label 1 sentences))
    repeats = list(map(lambda i : l.count(i),tokens))
    
    for i in range(0,len(repeats)):
        probe *= (repeats[i] + 1)/(unique + label_words)
    return probe
def probability_label2(p,tokens,l,unique,label_words):
    probe = p 
    repeats = list(map(lambda i : l.count(i),tokens))
    
    for i in range(0,len(repeats)):
        probe *= (repeats[i] + 1)/(unique + label_words)
    return probe
def probability_label3(p,tokens,l,unique,label_words):
    probe = p 
    repeats = list(map(lambda i : l.count(i),tokens))
    
    for i in range(0,len(repeats)):
        probe *= (repeats[i] + 1)/(unique + label_words)
    return probe
def probability_label4(p,tokens,l,unique,label_words):
    probe = p 
    repeats = list(map(lambda i : l.count(i),tokens))
    
    for i in range(0,len(repeats)):
        probe *= (repeats[i] + 1)/(unique + label_words)
    return probe
def probability_label5(p,tokens,l,unique,label_words):
    probe = p 
    repeats = list(map(lambda i : l.count(i),tokens))
    
    for i in range(0,len(repeats)):
        probe *= (repeats[i] + 1)/(unique + label_words)
    return probe
def predict(sentence,tr,l1,l2,l3,l4,l5):
    ################### now we find probability of class(label) of each sentence ##################
    p1 = len(l1['query']) / len(tr['query']) 
    p2 = len(l2['query']) / len(tr['query'])
    p3 = len(l3['query']) / len(tr['query'])
    p4 = len(l4['query']) / len(tr['query'])
    p5 = len(l5['query']) / len(tr['query'])

    ##########################also we need to find how many unique words we have in all train sentences##################
    all_sentences = df['query'][0]
    all_sentences = " ".join(df['query'][1:len(df['query'])])
    all_sentences = tokenize(all_sentences)
    all_sentences = clean(all_sentences)
    unique_word_tokens = set(all_sentences)
    unique = len(unique_word_tokens)
    
    
    ##################################### we need all word tokens number in each label's sentences ##################
    l1_sentences = " ".join(l1['query'][0:len(l1['query'])])
    l2_sentences = " ".join(l2['query'][0:len(l2['query'])])
    l3_sentences = " ".join(l3['query'][0:len(l3['query'])])
    l4_sentences = " ".join(l4['query'][0:len(l4['query'])])
    l5_sentences = " ".join(l5['query'][0:len(l5['query'])])
    
    l1_sentences = tokenize(l1_sentences)
    l1_sentences = clean(l1_sentences)
    label1_words = len(l1_sentences)
    l2_sentences = tokenize(l2_sentences)
    l2_sentences = clean(l2_sentences)
    label2_words = len(l2_sentences)
    l3_sentences = tokenize(l3_sentences)
    l3_sentences = clean(l3_sentences)
    label3_words = len(l3_sentences)
    l4_sentences = tokenize(l4_sentences)
    l4_sentences = clean(l4_sentences)
    label4_words = len(l4_sentences)
    l5_sentences = tokenize(l5_sentences)
    l5_sentences = clean(l5_sentences)
    label5_words = len(l5_sentences)
    
    ############################# now we merge all tokens into one list for finding each word repeats in each label ##################
    label1_tokens,label2_tokens,label3_tokens,label4_tokens,label5_tokens = [],[],[],[],[]
    
    list_1 = list(l1['tokens'])
    for i in list_1 :
        label1_tokens.extend(i)
    
    list_2 = list(l2['tokens'])
    for i in list_2 :
        label2_tokens.extend(i)
        
    list_3 = list(l3['tokens'])
    for i in list_3 :
        label3_tokens.extend(i)
        
    list_4 = list(l4['tokens'])
    for i in list_4 :
        label4_tokens.extend(i)
        
    list_5 = list(l5['tokens'])
    for i in list_5 :
        label5_tokens.extend(i)    
    
    ##################################### these are constant numbers during naive bayes algorithm  ####################################  
    
    label_1 = probability_label1(p1,sentence,label1_tokens,unique,label1_words)
    label_2 = probability_label2(p2,sentence,label2_tokens,unique,label2_words)
    label_3 = probability_label3(p3,sentence,label3_tokens,unique,label3_words)
    label_4 = probability_label4(p4,sentence,label4_tokens,unique,label4_words)
    label_5 = probability_label5(p5,sentence,label5_tokens,unique,label5_words)
    
    maximum = 0
    label = 0
    if label_1> maximum:
        maximum = label_1
        label = 1
    if label_2> maximum:
        maximum = label_2
        label = 2
    if label_3> maximum:
        maximum = label_3
        label = 3
    if label_4> maximum:
        maximum = label_4
        label = 4
    if label_5> maximum:
        maximum = label_5
        label = 5
    return label;


# K-fold function

### حال تابع کا-فولد را داریم که ورودی اول آن برای تست های کا-فولد مهم نیست
### چرا که در اصل همان داده تستی است که برای ارسال در سایت کگل نیاز داریم
### برای تشخیص اینکه کا-فولد میخواهیم یا تست کگل، جهت جلوگیری از تکرار
### کد، یک پارامتر به نام الگوریتم دارم که اگر یک بود یعنی کا-فولد و اگر برابر
### با صفر بود یعنی تست کگل دارد انجام می شود. سپس داده دیتافریم که برای 
### کا-فولد معادل کل داده ها است و باید آموزش و تست را انتخاب کنیم ازش و
### برای تست کگل برابر است با خود داده آموزش. ابتدا هر کوئری در داده آموزش
### و تست نرمالایز و توکن بندی می شود و سپس با تمیز کردن توکن ها و تقسیم
### بندی داده های آموزشی به 5 دیتافریم که هر دیتافریم مرتبط با گفته های یک 
### .لیبل است به سراغ انجام الگوریتم بیز ضعیف می رویم

In [6]:
from sklearn.metrics import precision_recall_fscore_support
def k_fold(test_main,df,k,alg):


    if(alg == 1):#we want k-fold
        ################### based on 3-fold way we should ##################
        if  (k == 1):
            train_set = df[0:2031]
            test_set = df[2032:3047]
        elif(k == 2):
            train_set = df[0:1015]
            train_set.append(df[2032:3047])
            test_set = df[1016:2031]
        else:
            train_set = df[1016:3047]
            test_set = df[0:1015]
    else:################### we work on main test set ##################
        train_set = df
        test_set = test_main
    ################### we should give indexes to our train set ##################
    train_set['index'] = pd.Series(range(0,len(train_set['query']),1)).values
######now we normalize and tokenize all sentences in train set and test set also we add a new column named "tokens" includes list of tokens of each sentence
    train_set['tokens'] = train_set['query'].apply(tokenize)
    test_set['tokens'] = test_set['query'].apply(tokenize)


################### now we clean the data, which is removing question marks, stop point, conjuctive words,
################### pronouns, some special verbs and others which are included in a list named "extra"
    train_set['tokens'] = train_set['tokens'].apply(clean)
    test_set['tokens'] = test_set['tokens'].apply(clean)
    ##### show them after cleaning #######
    print('-------train set after tokenizing and cleaning data is shown below-------') 
    display(train_set)
#     train_set.to_csv('train_after_process.csv')
    print('-------test set after tokenizing and cleaning data is shown below-------')
    display(test_set)
##################################### at last we divide train samples into 5 label based dataframes ##################
    train_lab1, train_lab2, train_lab3, train_lab4, train_lab5 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(),pd.DataFrame()
    
    train_lab1 = train_set[train_set['label'] == 1].set_index('label')
    train_lab2 = train_set[train_set['label'] == 2].set_index('label')
    train_lab3 = train_set[train_set['label'] == 3].set_index('label')
    train_lab4 = train_set[train_set['label'] == 4].set_index('label')
    train_lab5 = train_set[train_set['label'] == 5].set_index('label')

    ################### we need a new column named index which starts from 0 to length of each label based dataframe ##################
    train_lab1['index'] = pd.Series(range(0,len(train_lab1['tokens']),1)).values
    train_lab2['index'] = pd.Series(range(0,len(train_lab2['tokens']),1)).values
    train_lab3['index'] = pd.Series(range(0,len(train_lab3['tokens']),1)).values
    train_lab4['index'] = pd.Series(range(0,len(train_lab4['tokens']),1)).values
    train_lab5['index'] = pd.Series(range(0,len(train_lab5['tokens']),1)).values

    ################### now we should iterate on test set and predict each sentence's label
    ################### we pass the current sentence into a function to predict and return a label
    ################### our procedure is based on modular programming
    test_set['prediction'] = test_set['tokens'].apply(predict,tr = train_set,l1 = train_lab1,l2 = train_lab2,l3 = train_lab3,l4 = train_lab4,l5 = train_lab5)

    
    ###################  at the last part we need to find out percision score by two standards of fscore and percision rate (which is self described :) and equals to number of true predictions by all predictions) 
    rates = precision_recall_fscore_support(test_set['label'],test_set['prediction'])
    print('Fscore details: {}'.format(rates))
    print('Percision for this test is : {}'.format(len(test_set[test_set['prediction'] == test_set['label']]) / len(test_set['query'])))
    
    return test_set

# K-fold tests for K = 1,2,3

### سه مرتبه الگوریتم کا-فولد را اجرا می کنیم و هر سری دقت پیشبینی را بررسی می کنیم
### سپس جهت بررسی یکبار بدون تمیزکاری و البته نرمالایز کردن داده ها این الگوریتم را
### روی مرحله اول که کا معادل یک بود اجرا می کنیم و دقت آن را نیز با حالتی که نرمالایز
### .و تمیزکاری دیتا کمک کرده اند بررسی کرده ایم

In [7]:
# test_1,test_2,test_3 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame() 
# print('test for 1-fold')
# test_1 = k_fold(df,df,1,1)
# print('-------dataframe after processing and predicting data is shown below-------')
# display(test_1)    
# print('test for 2-fold')
# test_2 = k_fold(df,df,2,1)
# print('test for 3-fold')
# test_3 = k_fold(df,df,3,1)

### !!!!!!!!! در ادامه تست بدون نورمالایز و البته تمیزشدن دیتا انجام می شود که دقت بالاتری پیدا می کند نسبت به حالت قبل
# دلیل خطا در دقت
### بنظرم عدمت توزیع مناسب جمله ها در داده های تست و آموزش می تواند موجب این خطا بشود، نبودن تنوع میان جملات
### می توانست تکرار در لیبل ها را زیاد کند و این امر موجب افزایش دقت در حالتی است که حروف اضافه و غیره حذف نشدن

In [8]:
# print('test for 1-fold without normalizing and cleaning data')
# test_1_withNoNormalizeAndClean = k_fold(df,df,1,1)
# print('-------dataframe after processing and predicting data is shown below-------')
# display(test_1_withNoNormalizeAndClean)

# Final test

### حال ما تابع طراحی شده را یکبار با قراردادن متغیر صفر بعنوان آخرین پارامترش صدا میزنیم البته نام کا-فولد دیگر مناسب نیست اما برای جلوگیری از تکرار کد همین تابع را استفاده کرده ام تا روی مجموعه آموزش و تست اصلی جهت ارسال در سایت کگل کار کنم
#### :)

In [9]:
test_main = pd.read_csv('inference.csv')
final = k_fold(test_main,df,0,0)
final.to_csv('result.csv')
print('-------final test after prediction is shown as below-------')
display(final)

-------train set after tokenizing and cleaning data is shown below-------


,id,query,label,index,tokens
0,0,شرایط حذف ترم چیه؟,1,0,"[شرایط, حذف, ترم]"
1,1,از کجا می تونم با دکتر وحیدی ارتباط برقرار کنم؟,2,1,"[کجا, می‌تونم, دکتر, وحیدی, ارتباط, برقرار, کنم]"
2,2,بوفه برداران تا ساعت چند باز است؟,2,2,"[بوفه, برداران, ساعت, باز]"
3,3,کمترین تعداد واحد چند عدد است؟,1,3,"[کمترین, تعداد, واحد, عدد]"
4,4,سنگ جامد است,5,4,"[سنگ, جامد]"
...,...,...,...,...,...
3043,3043,چند تا درس میشه حذف کرد,1,3043,"[درس, میشه, حذف, کرد]"
3044,3044,جدید ترین ویرایش کتاب هریس که موجوده چیه؟,3,3044,"[جدید‌ترین, ویرایش, کتاب, هریس, که, موجوده]"
3045,3045,شرایط مهمان شدن در دانشکده ما چیست؟,1,3045,"[شرایط, مهمان, شدن, در, دانشکده, چیست]"
3046,3046,آمفی تئاتر دانشکده کامپیوتر کجاست؟,2,3046,"[آمفی, تئاتر, دانشکده, کامپیوتر, کجاست]"


-------test set after tokenizing and cleaning data is shown below-------


,Unnamed: 0,query,label,tokens
0,0,آیا درس فلان همیشه توسط یک استاد ارایه میشود؟,1,"[درس, فلان, همیشه, توسط, یک, استاد, ارایه, میشود]"
1,1,آیا میشه درسی از ارشد برداشت؟,1,"[میشه, درسی, ارشد, برداشت]"
2,2,لطفاً اختیاری های بهتری ارايه کنید ما انتخابی ...,4,"[لطفا, اختیاری‌های, بهتری, ارایه, کنید, انتخاب..."
3,3,آیا میتوانیم درس شبکه های عصبی ارشد را برداریم؟,1,"[میتوانیم, درس, شبکه‌های, عصبی, ارشد, برداریم]"
4,4,هیچکس در دانشگاه واسخگوی مطالبات دانشجویان نیست,4,"[هیچکس, در, دانشگاه, واسخگوی, مطالبات, دانشجویان]"
5,5,آیا سیاست های جدید دولت بر قیمت نفت تاثیر دارد؟,5,"[سیاست‌های, جدید, دولت, بر, قیمت, نفت, تاثیر, ..."
6,6,چرا سرور دانشکده پایین است؟,3,"[سرور, دانشکده, پایین]"
7,7,لطفا به من درباره دانشکده توضیح مختصری بدهید.,2,"[لطفا, درباره, دانشکده, توضیح, مختصری, بدهید]"
8,8,لطفا نمره ی پروژه ی درس برنامه نویسی بیشتر شود.,4,"[لطفا, نمره‌ی, پروژه‌ی, درس, برنامه, نویسی, بی..."
9,9,هوا هفته ی آینده بارانی است.,5,"[هوا, هفته‌ی, آینده, بارانی]"


Fscore details: (array([0.8125    , 0.66666667, 0.83333333, 0.4       , 0.77777778]), array([0.76470588, 0.6       , 0.83333333, 0.8       , 0.58333333]), array([0.78787879, 0.63157895, 0.83333333, 0.53333333, 0.66666667]), array([17, 10,  6,  5, 12], dtype=int64))
Percision for this test is : 0.7
-------final test after prediction is shown as below-------


,Unnamed: 0,query,label,tokens,prediction
0,0,آیا درس فلان همیشه توسط یک استاد ارایه میشود؟,1,"[درس, فلان, همیشه, توسط, یک, استاد, ارایه, میشود]",4
1,1,آیا میشه درسی از ارشد برداشت؟,1,"[میشه, درسی, ارشد, برداشت]",1
2,2,لطفاً اختیاری های بهتری ارايه کنید ما انتخابی ...,4,"[لطفا, اختیاری‌های, بهتری, ارایه, کنید, انتخاب...",4
3,3,آیا میتوانیم درس شبکه های عصبی ارشد را برداریم؟,1,"[میتوانیم, درس, شبکه‌های, عصبی, ارشد, برداریم]",1
4,4,هیچکس در دانشگاه واسخگوی مطالبات دانشجویان نیست,4,"[هیچکس, در, دانشگاه, واسخگوی, مطالبات, دانشجویان]",5
5,5,آیا سیاست های جدید دولت بر قیمت نفت تاثیر دارد؟,5,"[سیاست‌های, جدید, دولت, بر, قیمت, نفت, تاثیر, ...",5
6,6,چرا سرور دانشکده پایین است؟,3,"[سرور, دانشکده, پایین]",3
7,7,لطفا به من درباره دانشکده توضیح مختصری بدهید.,2,"[لطفا, درباره, دانشکده, توضیح, مختصری, بدهید]",4
8,8,لطفا نمره ی پروژه ی درس برنامه نویسی بیشتر شود.,4,"[لطفا, نمره‌ی, پروژه‌ی, درس, برنامه, نویسی, بی...",4
9,9,هوا هفته ی آینده بارانی است.,5,"[هوا, هفته‌ی, آینده, بارانی]",5
